In [20]:
# add modules folder to Python's search path
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
%matplotlib inline
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import pandas as pd
import time
import arch
import utility as ut

tfd = tfp.distributions

C:\Users\pinak\Documents\GitHub\fp-solvers


**Define $p_0,\mu, p_\infty$**

In [23]:
DTYPE = 'float32'
save_folder = '../non-grad3D/L63'
dt = 0.01
n_steps = 1
root2 = np.sqrt(2., dtype= DTYPE)
dim = 3
sigma = 10.
D = sigma**2 / 2.
l = np.ones(dim, dtype=DTYPE)
g1 = tfd.MultivariateNormalDiag(loc=2.*l, scale_diag=l)
g2 = tfd.MultivariateNormalDiag(loc=-2.*l, scale_diag=l)
mix = 0.5
rv0 = tfd.Mixture(cat=tfd.Categorical(probs=[mix, 1.-mix]), components=[g1, g2])
log_p0 = lambda x: tf.reshape(rv0.log_prob(x), (-1, 1))


net = arch.LSTMForgetNet(50, 3, DTYPE, name="L63")
net.load_weights('../non-grad3D/data/L63/1M/{}'.format(net.name)).expect_partial()

alpha, beta, rho = 10., 8./3., 28.

def mu(X):
    x, y, z = np.split(X, dim, axis=-1)
    p = alpha * (y - x) 
    q = x * (rho - z) - y 
    r = x * y - beta * z
    return np.concatenate([p, q, r], axis=-1)


def h0(X):
    return np.exp(log_p0(X) - net(X))


def get_endpts(dim, n_repeats):
    X = np.ones((n_repeats, dim), dtype=DTYPE) / root2
    dW = np.random.normal(scale=np.sqrt(dt), size=(n_steps, n_repeats, dim)).astype(DTYPE)
    for step in range(n_steps):
        X += mu(X) * dt + sigma * dW[step]
    return X

@ut.timer
def fk(n_repeats, m=1):
    x0 = np.ones((1, dim), dtype=DTYPE) / root2
    val = 0.
    for i in range(m):
      X = get_endpts(dim, n_repeats)
      val += np.exp(net(x0))[0, 0] * (h0(X).sum()) / n_repeats
    return val/m

def solve():
    print("Solving for d = {}".format(dim))
    n_particles = [1, 100, 10000, 1000000, 2000000]
    sol = np.zeros(len(n_particles))
    for i, n in enumerate(n_particles):
      sol[i] = fk(n)
    pd.DataFrame({'N': n_particles, 'sol': sol}).to_csv('{}/sol.csv'.format(save_folder), index=None)
    return sol

def get_sol(dim):
    df = pd.read_csv('{}/sol.csv'.format(save_folder))
    return df['N'].to_numpy(), df['sol'].to_numpy()

def plot():
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    n, s = get_sol(dim)
    e = np.abs(s-s[-1])
    ax.plot(np.log10(n[:-1]), np.log10(e[:-1]), label='d={}'.format(dim))
    ax.legend()
    plt.show()

In [24]:
start = time.time()
solve()
plot(dim)
end = time.time()
print('Time taken by this cell = {}'.format(end-start))

Solving for d = 3
Time taken by fk is 0.08208918571472168 seconds
Time taken by fk is 0.02601337432861328 seconds
Time taken by fk is 0.06358814239501953 seconds


ResourceExhaustedError: Exception encountered when calling layer "LSTMForgetBlock" (type LSTMForgetBlock).

failed to allocate memory [Op:AddV2]

Call arguments received:
  • x=tf.Tensor(shape=(1000000, 3), dtype=float32)
  • h=tf.Tensor(shape=(1000000, 3), dtype=float32)
  • c=tf.Tensor(shape=(1000000, 50), dtype=float32)